# Feature Extraction

Trying out 2 ways of dealing with data:
- Option 1 : save croped images and skeleton into pickle files
- Option 2 : save croped images and skeleton into a unique h5 file

In [1]:
# Dataset path 
ntu_path = "/home/gnocchi/Documents/datasets/NTU-RGB-D_toy/"
rgb_folder = "nturgb+d_rgb/"
skeleton_folder = "nturgb+d_skeletons/"

# Hyper parameters
crop_size = 50
offset = int(crop_size / 2)

import numpy as np
import os
import time

import skvideo.io
import skvideo.datasets

import pickle
import h5py

# Custom imports
from utils import * 
from joints import *



## Pickle approach 

In [2]:
start = time.time()

# Loop through skeleton files
for filename in os.listdir(ntu_path + skeleton_folder):
    # Retrieve skeleton data
    skeleton = read_xyz(ntu_path + skeleton_folder + filename) # shape (2, max_frame, num_joint=25, n_subjects)
    skeleton_rgb = read_color_xy(ntu_path + skeleton_folder + filename) # shape (2, max_frame, num_joint=25, n_subjects)
    
    # Assert skeletons are correct
    assert skeleton.shape[1:] == skeleton_rgb.shape[1:]
    
    # Sequence code without extension
    short_filename = os.path.splitext(filename)[0]
    print(short_filename)
        
    # Read cooresponding video
    videodata = skvideo.io.vread(ntu_path + rgb_folder + short_filename +'_rgb.avi') # shape (n_frames, 1080, 1920, 3)
    
    # Check that video data has same number of frames as skeleton
    assert skeleton.shape[1] == videodata.shape[0]
    
    n_frames = videodata.shape[0]
    
    # Create empty np arrays containing crops of hands from videos
    # shape (n_frames, number of hands (2 subjects), x, y)
    hand_crops = np.zeros((n_frames, 4, crop_size, crop_size, 3), dtype=np.uint8)
    
    left_hand_s0_x = int(skeleton_rgb[1, 0, Joints.HANDLEFT, 0])
    left_hand_s0_y = int(skeleton_rgb[0, 0, Joints.HANDLEFT, 0])
    
    right_hand_s0_x = int(skeleton_rgb[1, 0, Joints.HANDRIGHT, 0])
    right_hand_s0_y = int(skeleton_rgb[0, 0, Joints.HANDRIGHT, 0])
    
    left_hand_s1_x = int(skeleton_rgb[1, 0, Joints.HANDLEFT, 1])
    left_hand_s1_y = int(skeleton_rgb[0, 0, Joints.HANDLEFT, 1])
    
    right_hand_s1_x = int(skeleton_rgb[1, 0, Joints.HANDRIGHT, 1])
    right_hand_s1_y = int(skeleton_rgb[0, 0, Joints.HANDRIGHT, 1])
    
    hand_crops[:, 0] = videodata[:, 
                                 left_hand_s0_x-offset:left_hand_s0_x+offset, 
                                 left_hand_s0_y-offset:left_hand_s0_y+offset, 
                                 :]
    hand_crops[:, 1] = videodata[:, 
                                 right_hand_s0_x-offset:right_hand_s0_x+offset, 
                                 right_hand_s0_y-offset:right_hand_s0_y+offset, 
                                 :]
    if right_hand_s1_x + right_hand_s1_y + left_hand_s1_x + left_hand_s1_y != 0:
        hand_crops[:, 2] = videodata[:, 
                                     left_hand_s1_x-offset:left_hand_s1_x+offset, 
                                     left_hand_s1_y-offset:left_hand_s1_y+offset, 
                                     :]
        hand_crops[:, 3] = videodata[:, 
                                     right_hand_s1_x-offset:right_hand_s1_x+offset, 
                                     right_hand_s1_y-offset:right_hand_s1_y+offset, 
                                     :]
    with open('pickles/' + short_filename + '_skeleton.pkl', 'wb') as pickle_file:
        pickle.dump(skeleton, pickle_file)
        
    with open('pickles/' + short_filename + '_rgb.pkl', 'wb') as pickle_file:
        pickle.dump(hand_crops, pickle_file)
        
end = time.time()
print("It took : " + str(end - start) + " seconds")

S001C001P001R001A039
S001C001P001R001A052
S001C001P001R001A031
S001C001P001R001A023
S001C001P001R001A017
S001C001P001R001A056
S001C001P001R001A043
S001C001P001R001A007
S001C001P001R001A009
S001C001P001R001A057
S001C001P001R001A015
S001C001P001R001A029
S001C001P001R001A014
S001C001P001R001A002
S001C001P001R001A020
S001C001P001R001A006
S001C001P001R001A047
S001C001P001R001A046
S001C001P001R001A038
S001C001P001R001A011
S001C001P001R001A058
S001C001P001R001A025
S001C001P001R001A001
S001C001P001R001A040
S001C001P001R001A051
S001C001P001R001A045
S001C001P001R001A016
S001C001P001R001A048
S001C001P001R001A030
S001C001P001R001A042
S001C001P001R001A024
S001C001P001R001A019
S001C001P001R001A010
S001C001P001R001A059
S001C001P001R001A012
S001C001P001R001A005
S001C001P001R001A033
S001C001P001R001A026
S001C001P001R001A053
S001C001P001R001A034
S001C001P001R001A041
S001C001P001R001A035
S001C001P001R001A055
S001C001P001R001A022
S001C001P001R001A054
S001C001P001R001A003
S001C001P001R001A060
S001C001P001R

## h5py approach

Takes same time and same space without compression. 

Takes longer (~15s per 60 samples) when compressing with GZIP, compression_opts=9 but saves ~115MB per 60 samples.

Takes longer (4s per 60 samples) when compression with Lzf, but saves 90MB


In [3]:
start = time.time()

with h5py.File('datasets.h5', 'w') as hdf:
    # Loop through skeleton files
    for filename in os.listdir(ntu_path + skeleton_folder):
        # Retrieve skeleton data
        skeleton = read_xyz(ntu_path + skeleton_folder + filename) # shape (2, max_frame, num_joint=25, n_subjects)
        skeleton_rgb = read_color_xy(ntu_path + skeleton_folder + filename) # shape (2, max_frame, num_joint=25, n_subjects)

        # Assert skeletons are correct
        assert skeleton.shape[1:] == skeleton_rgb.shape[1:]

        # Sequence code without extension
        short_filename = os.path.splitext(filename)[0]
        print(short_filename)

        # Read cooresponding video
        videodata = skvideo.io.vread(ntu_path + rgb_folder + short_filename +'_rgb.avi') # shape (n_frames, 1080, 1920, 3)

        # Check that video data has same number of frames as skeleton
        assert skeleton.shape[1] == videodata.shape[0]

        n_frames = videodata.shape[0]

        # Create empty np arrays containing crops of hands from videos
        # shape (n_frames, number of hands (2 subjects), x, y)
        hand_crops = np.zeros((n_frames, 4, crop_size, crop_size, 3), dtype=np.uint8)

        left_hand_s0_x = int(skeleton_rgb[1, 0, Joints.HANDLEFT, 0])
        left_hand_s0_y = int(skeleton_rgb[0, 0, Joints.HANDLEFT, 0])

        right_hand_s0_x = int(skeleton_rgb[1, 0, Joints.HANDRIGHT, 0])
        right_hand_s0_y = int(skeleton_rgb[0, 0, Joints.HANDRIGHT, 0])

        left_hand_s1_x = int(skeleton_rgb[1, 0, Joints.HANDLEFT, 1])
        left_hand_s1_y = int(skeleton_rgb[0, 0, Joints.HANDLEFT, 1])

        right_hand_s1_x = int(skeleton_rgb[1, 0, Joints.HANDRIGHT, 1])
        right_hand_s1_y = int(skeleton_rgb[0, 0, Joints.HANDRIGHT, 1])

        hand_crops[:, 0] = videodata[:, 
                                     left_hand_s0_x-offset:left_hand_s0_x+offset, 
                                     left_hand_s0_y-offset:left_hand_s0_y+offset, 
                                     :]
        hand_crops[:, 1] = videodata[:, 
                                     right_hand_s0_x-offset:right_hand_s0_x+offset, 
                                     right_hand_s0_y-offset:right_hand_s0_y+offset, 
                                     :]
        if right_hand_s1_x + right_hand_s1_y + left_hand_s1_x + left_hand_s1_y != 0:
            hand_crops[:, 2] = videodata[:, 
                                         left_hand_s1_x-offset:left_hand_s1_x+offset, 
                                         left_hand_s1_y-offset:left_hand_s1_y+offset, 
                                         :]
            hand_crops[:, 3] = videodata[:, 
                                         right_hand_s1_x-offset:right_hand_s1_x+offset, 
                                         right_hand_s1_y-offset:right_hand_s1_y+offset, 
                                         :]
            
        sample = hdf.create_group(short_filename)
        sample.create_dataset("skeleton", data = skeleton, compression="gzip", compression_opts=9)
        sample.create_dataset("rgb", data = hand_crops, compression="gzip", compression_opts=9)
        
end = time.time()
print("It took : " + str(end - start) + " seconds")

S001C001P001R001A039
S001C001P001R001A052
S001C001P001R001A031
S001C001P001R001A023
S001C001P001R001A017
S001C001P001R001A056
S001C001P001R001A043
S001C001P001R001A007
S001C001P001R001A009
S001C001P001R001A057
S001C001P001R001A015
S001C001P001R001A029
S001C001P001R001A014
S001C001P001R001A002
S001C001P001R001A020
S001C001P001R001A006
S001C001P001R001A047
S001C001P001R001A046
S001C001P001R001A038
S001C001P001R001A011
S001C001P001R001A058
S001C001P001R001A025
S001C001P001R001A001
S001C001P001R001A040
S001C001P001R001A051
S001C001P001R001A045
S001C001P001R001A016
S001C001P001R001A048
S001C001P001R001A030
S001C001P001R001A042
S001C001P001R001A024
S001C001P001R001A019
S001C001P001R001A010
S001C001P001R001A059
S001C001P001R001A012
S001C001P001R001A005
S001C001P001R001A033
S001C001P001R001A026
S001C001P001R001A053
S001C001P001R001A034
S001C001P001R001A041
S001C001P001R001A035
S001C001P001R001A055
S001C001P001R001A022
S001C001P001R001A054
S001C001P001R001A003
S001C001P001R001A060
S001C001P001R